In [3]:
!pip3 install pyspark datasets evaluate accelerate -U
import numpy as np
import torch
from pyspark.sql import SparkSession
from pyspark.ml.torch.distributor import TorchDistributor
from transformers import Trainer, TrainingArguments
from transformers.training_args import OptimizerNames
from datasets import load_dataset
import evaluate
from transformers import AutoImageProcessor, AutoModelForImageClassification
# !wget -c https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
# !tar -xvzf cifar-10-python.tar.gz
dataset_path = 'dataset/'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 73.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=016a7cb6e409dc02b390418aca767615ede15046d886c3232a64f088525c70bb
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed

In [ ]:
def get_fine_tuning_trainer_args(output_path, hyperparameters=None):

    return TrainingArguments(
        output_dir=output_path + 'training/',
        logging_dir=output_path + 'logs/',
        per_device_train_batch_size=5,
        per_device_eval_batch_size=5,
        evaluation_strategy="steps",
        num_train_epochs=1,
        save_steps=40,
        eval_steps=40,
        logging_steps=1,
        learning_rate=5.e-05,
        warmup_ratio=0.1,
        warmup_steps=1,
        weight_decay=0,
        save_total_limit=2,
        metric_for_best_model='accuracy',
        greater_is_better=True,
        optim=OptimizerNames.ADAMW_HF,
        remove_unused_columns=False,
        push_to_hub=False,
        load_best_model_at_end=True,
        seed=42,
        gradient_accumulation_steps=4,
    )

def build_metrics():

    _ = evaluate.load("accuracy", cache_dir="metrics/", trust_remote_code=True)

    metric = evaluate.combine(["accuracy"])

    def compute_metrics(p):
        return metric.compute(
            predictions=np.argmax(p.predictions, axis=1),
            references=p.label_ids
        )

    return compute_metrics

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [84]:
def startTraining(model_name, distributed_training=False):

    train_dataset = load_dataset('cifar10', split=f"train[:5]", verification_mode='no_checks',
                     cache_dir=dataset_path)

    test_dataset = load_dataset('cifar10', split=f"test[:5]", verification_mode='no_checks',
                                     cache_dir=dataset_path)

    pretrained_model = AutoModelForImageClassification.from_pretrained(model_name, cache_dir='models/', num_labels=10, ignore_mismatched_sizes=True)

    feature_extractor = AutoImageProcessor.from_pretrained(model_name, cache_dir='models/')

    def trainer():
        print(f"Starting fine-tuning on model {model_name}:")


        def preprocess(batchImage):
            inputs = feature_extractor(batchImage['img'], return_tensors='pt')
            inputs['label'] = batchImage['label']
            return inputs

        fine_tune_args = get_fine_tuning_trainer_args("results/")

        # print(fine_tune_args.device)

        fine_tune_trainer = Trainer(
            model=pretrained_model,
            args=fine_tune_args,
            data_collator=collate_fn,
            compute_metrics=build_metrics(),
            train_dataset=train_dataset.with_transform(preprocess),
            eval_dataset=test_dataset.with_transform(preprocess),
        )

        train_results = fine_tune_trainer.train()

        fine_tune_trainer.save_model(output_dir='results/models')

        fine_tune_trainer.log_metrics("train", train_results.metrics)
        fine_tune_trainer.save_state()

        metrics = fine_tune_trainer.evaluate(test_dataset.with_transform(preprocess))
        fine_tune_trainer.log_metrics("eval", metrics)
        fine_tune_trainer.save_metrics("eval", metrics)

        torch.distributed.destroy_process_group()
        return train_results


    if distributed_training:
        import torch.distributed
        torch.distributed.init_process_group(backend="nccl")
        spark = SparkSession.builder. \
          appName("ImageClassification"). \
          master("local[*]"). \
          config("spark.executor.memory", "16G"). \
          config("spark.driver.memory", "16G"). \
          config("spark.executor.resource.gpu.amount", "14").\
          config("spark.driver.resource.gpu.amount","1").\
          config("spark.task.resource.gpu.amount","1").\
          config("spark.executor.resource.gpu.discoveryScript", "/content/dis.sh").\
          config("spark.driver.resource.gpu.discoveryScript", "/content/dis.sh").\
          getOrCreate()
        NUM_PROCESSES = torch.cuda.device_count()
        results = TorchDistributor(num_processes=NUM_PROCESSES, local_mode=True, use_gpu=True).run(trainer)
        spark.stop()
    else:
      trainer()

In [73]:
startTraining('facebook/deit-base-patch16-224', False)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting fine-tuning on model facebook/deit-base-patch16-224:


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
40,0.310600,0.247193,0.940700
80,0.157800,0.144176,0.960800


***** train metrics *****
  epoch                    =      0.9923
  total_flos               = 896127456GF
  train_loss               =      0.5092
  train_runtime            =  0:12:29.83
  train_samples_per_second =       16.67
  train_steps_per_second   =       0.129


***** eval metrics *****
  epoch                   =     0.9923
  eval_accuracy           =     0.9608
  eval_loss               =     0.1442
  eval_runtime            = 0:02:18.74
  eval_samples_per_second =     72.074
  eval_steps_per_second   =      2.256


In [74]:
startTraining('facebook/deit-base-patch16-224', True)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:TorchDistributor:Started local training with 1 processes


2024-04-20 19:53:17.984582: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 19:53:17.984635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 19:53:17.986121: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-20 19:53:19.075982: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Starting fine-tuning on model facebook/deit-base-patch16-224:
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use th

INFO:TorchDistributor:Finished local training with 1 processes


100%|██████████| 313/313 [02:19<00:00,  2.25it/s]
***** eval metrics *****
  epoch                   =     0.9923
  eval_accuracy           =     0.9602
  eval_loss               =     0.1411
  eval_runtime            = 0:02:19.94
  eval_samples_per_second =     71.457
  eval_steps_per_second   =      2.237
